In [1]:
# data downloading
import data_util
import pandas as pd
from collections import defaultdict
import os
import sys

os.chdir("/home/yxjiang/source/ml_playground")

print(os.getcwd())

dataset_url="https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"

dest_dir = "/tmp/data"
dataset_folder_path = os.path.join(dest_dir, "aclImdb")
data_util.download_data(url=dataset_url, dest_dir=dest_dir)

# generate word to id mapping
word_to_id, word_list = data_util.get_vocabulary(folder_path=dataset_folder_path, file_suffix="vocab")
print("There size of vocabulary is :", len(word_to_id))


/home/yxjiang/source/ml_playground
Destination folder [/tmp/data] exists.
Target file [aclImdb_v1.tar.gz] exists, skip downloading.
Start to extract [/tmp/data/aclImdb_v1.tar.gz] to [/tmp/data]...
File extracted
Processing vocabulary from [/tmp/data/aclImdb].
There size of vocabulary is : 89527


In [3]:
# put everything together
import time
from data_util import *
from models import *
from trainer import classification_trainer
from config import TextCNNConfig
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F

config = TextCNNConfig(
            criteria=nn.CrossEntropyLoss, optimizer=optim.Adam, lr=0.00003, epochs=1000, 
            batch_size=1024, num_classes=2, sentence_max_length=30, word_embedding_length=128, 
            activation=F.relu, dropout=0.1, conv_layer_sizes=[3,4,5,6,7]
        )

pos_train_data_folder = os.path.join(dataset_folder_path, "train/pos")
neg_train_data_folder = os.path.join(dataset_folder_path, "train/neg")
train_dataset = MovieReviewDataset(config, pos_train_data_folder, neg_train_data_folder, word_to_id, 
                            transform=transforms.Compose([
                                CutOrPadTransform(config), WordsToIdsTransform(config, word_to_id)
                            ]))

pos_test_data_folder = os.path.join(dataset_folder_path, "test/pos")
neg_test_data_folder = os.path.join(dataset_folder_path, "test/neg")
test_dataset = MovieReviewDataset(config, pos_test_data_folder, neg_test_data_folder, word_to_id, 
                            transform=transforms.Compose([
                                CutOrPadTransform(config), WordsToIdsTransform(config, word_to_id)
                            ]))

train_dataloader = DataLoader(dataset=train_dataset, batch_size=config.batch_size, shuffle=True)
test_dataloader = DataLoader(dataset=test_dataset, batch_size=config.batch_size)

# model = TextCNN(config, len(word_to_id)).to(device)
model = RNN(config, len(word_list)).to(config.device)

classification_trainer.train(model, config, train_dataloader, test_dataloader)

[0 seconds](epoch: 1/1000)[1024 samples] loss: 0.691.
eval loss: 0.698, accuracy: 50.056% [12514/25000]


KeyboardInterrupt: 